In [1]:
from models.flownet2.networks.FlowNetC import FlowNetC
from models.flownet2.utils import tools
from models.flownet2 import models, losses

from utils.preprocessing import preprocessing_flownet, preprocessing_pwc

import torch
import argparse


In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--start_epoch', type=int, default=1)
parser.add_argument('--total_epochs', type=int, default=10000)
parser.add_argument('--batch_size', '-b', type=int, default=8, help="Batch size")
parser.add_argument('--train_n_batches', type=int, default=-1,
                    help='Number of min-batches per epoch. If < 0, it will be determined by training_dataloader')
parser.add_argument('--crop_size', type=int, nargs='+', default=[256, 256],
                    help="Spatial dimension to crop training samples for training")
parser.add_argument('--gradient_clip', type=float, default=None)
parser.add_argument('--schedule_lr_frequency', type=int, default=0, help='in number of iterations (0 for no schedule)')
parser.add_argument('--schedule_lr_fraction', type=float, default=10)
parser.add_argument("--rgb_max", type=float, default=255.)

parser.add_argument('--number_workers', '-nw', '--num_workers', type=int, default=8)
parser.add_argument('--number_gpus', '-ng', type=int, default=-1, help='number of GPUs to use')
parser.add_argument('--no_cuda', action='store_true')

parser.add_argument('--seed', type=int, default=1)
parser.add_argument('--name', default='run', type=str, help='a name to append to the save directory')
parser.add_argument('--save', '-s', default='./work', type=str, help='directory for saving')

parser.add_argument('--validation_frequency', type=int, default=5, help='validate every n epochs')
parser.add_argument('--validation_n_batches', type=int, default=-1)
parser.add_argument('--render_validation', action='store_true',
                    help='run inference (save flows to file) and every validation_frequency epoch')

parser.add_argument('--inference', action='store_true')
parser.add_argument('--inference_visualize', action='store_true',
                    help="visualize the optical flow during inference")
parser.add_argument('--inference_size', type=int, nargs='+', default=[-1, -1],
                    help='spatial size divisible by 64. default (-1,-1) - largest possible valid size would be used')
parser.add_argument('--inference_batch_size', type=int, default=1)
parser.add_argument('--inference_n_batches', type=int, default=-1)
parser.add_argument('--save_flow', action='store_true', help='save predicted flows to file')

parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--log_frequency', '--summ_iter', type=int, default=1, help="Log every n batches")

parser.add_argument('--skip_training', action='store_true')
parser.add_argument('--skip_validation', action='store_true')

parser.add_argument('--fp16', action='store_true', help='Run model in pseudo-fp16 mode (fp16 storage fp32 math).')
parser.add_argument('--fp16_scale', type=float, default=1024.,
                    help='Loss scaling, positive power of 2 values can improve fp16 convergence.')

tools.add_arguments_for_module(parser, models, argument_for_class='model', default='FlowNet2')

tools.add_arguments_for_module(parser, losses, argument_for_class='loss', default='L1Loss')

tools.add_arguments_for_module(parser, torch.optim, argument_for_class='optimizer', default='Adam', skip_params=['params'])

args, unknown = parser.parse_known_args()


In [3]:
flowC = FlowNetC(args).cuda()
flowC.eval()
print(torch.load("models/flownet2/weights/FlowNet2-C_checkpoint.pth.tar")['state_dict'].keys())
#flowC.load_state_dict(torch.load("models/flownet2/weights/FlowNet2-C_checkpoint.pth.tar")['state_dict'])

odict_keys(['conv1.0.weight', 'conv1.0.bias', 'conv2.0.weight', 'conv2.0.bias', 'conv3.0.weight', 'conv3.0.bias', 'conv_redir.0.weight', 'conv_redir.0.bias', 'conv3_1.0.weight', 'conv3_1.0.bias', 'conv4.0.weight', 'conv4.0.bias', 'conv4_1.0.weight', 'conv4_1.0.bias', 'conv5.0.weight', 'conv5.0.bias', 'conv5_1.0.weight', 'conv5_1.0.bias', 'conv6.0.weight', 'conv6.0.bias', 'conv6_1.0.weight', 'conv6_1.0.bias', 'deconv5.0.weight', 'deconv5.0.bias', 'deconv4.0.weight', 'deconv4.0.bias', 'deconv3.0.weight', 'deconv3.0.bias', 'deconv2.0.weight', 'deconv2.0.bias', 'predict_flow6.weight', 'predict_flow6.bias', 'predict_flow5.weight', 'predict_flow5.bias', 'predict_flow4.weight', 'predict_flow4.bias', 'predict_flow3.weight', 'predict_flow3.bias', 'predict_flow2.weight', 'predict_flow2.bias', 'upsampled_flow6_to_5.weight', 'upsampled_flow6_to_5.bias', 'upsampled_flow5_to_4.weight', 'upsampled_flow5_to_4.bias', 'upsampled_flow4_to_3.weight', 'upsampled_flow4_to_3.bias', 'upsampled_flow3_to_2.weig

In [4]:
imgs = torch.load('Data/img.pth')
segs = torch.load('Data/seg.pth')

fix = 6; mov=3
fixed = imgs[fix:fix+1,:,:].float() /255
moving = imgs[mov:mov+1,:,:].float() /255

fixed_seg = segs[fix:fix+1, :,:].float().contiguous()
moving_seg = segs[mov:mov+1,:,:].float().contiguous()

#fixed = F.interpolate(fixed.unsqueeze(0), size=(100,100)).view(1,100,100)
#moving = F.interpolate(moving.unsqueeze(0), size=(100,100)).view(1,100,100)

#fixed_seg = F.interpolate(fixed_seg.unsqueeze(0), size=(100,100)).view(1,100,100)
#moving_seg = F.interpolate(moving_seg.unsqueeze(0), size=(100,100)).view(1,100,100)

C,h,w = fixed.shape

In [7]:
flow_in = preprocessing_flownet(fixed.reshape(h,w,C),moving.reshape(h,w,C))#.cuda()
flowC_in = preprocessing_pwc(fixed.reshape(h,w,C),moving.reshape(h,w,C)).cuda()

In [8]:
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA],
    with_flops=True
) as p:
    flowC(flowC_in)
print(p.key_averages().table(
    sort_by="self_cuda_time_total", row_limit=-1))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls        TFLOPS  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                aten::cudnn_convolution         0.54%       9.684ms        99.43%        1.775s      93.400ms       3.937ms        80.48%       3.937ms     207.211us            19            --  
                   volta_scudnn_128x64_relu_small_nn_v1         0.00%       0.000us         0.00%       0.000us       0.000us       3.578ms        73.1

If I read this correctly, it takes around 4.89ms for one pas and requires around 4 TFlops